In [1]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pkgutil
import spiketag

from spiketag.realtime import BMI,Binner
from spiketag.analysis import decoder
from spiketag.base import probe


import serial

In [2]:
# reading fet file
fet_path = 'fet.bin'
fet = np.memmap(fet_path, dtype=np.int32).reshape(-1, 8)
t = fet[:, 0]  # time
grp_id = fet[:, 1]  # group id
fets = fet[:, 2:6]  # feature extraction
spk_id = fet[:, 6]  # spike id
energy = fet[:, 7]  # energy

In [3]:
# check spiketag module
package = spiketag

print(f"Modules in {package.__name__}:")
for importer, modname, ispkg in pkgutil.iter_modules(package.__path__):
    print(f"{modname} (Package: {ispkg})")

Modules in spiketag:
analysis (Package: True)
base (Package: True)
command (Package: False)
core (Package: True)
fpga (Package: True)
mvc (Package: True)
probe (Package: True)
quality (Package: True)
realtime (Package: True)
res (Package: True)
spiketag (Package: False)
utils (Package: True)
view (Package: True)


# bmi

In [4]:
# send_signal_to_teensy()
def send_signal_to_teensy(bmi):
#     print("Signal")
    if bmi.TTLserial is not None:
        bmi.TTLserial.write(b'a')
        bmi.TTLserial.flush()

In [5]:
# reset_signal_to_teensy()
def reset_signal_to_teensy(bmi):
    if bmi.TTLserial is not None:
        bmi.TTLserial.write(b'b')
        bmi.TTLserial.flush()

In [7]:
# while True
def bmi_loop(bmi_func):
    while True:
        bmi_func()

In [8]:
# bmi laser function (mode)
def bmi_func(bmi, mode,targetID,neuron_id,threshold):
    frFlag = 0
    spkFlag = 0
    bmi_output = bmi.read_bmi()
    if mode != "spike ID":
        bmi.binner.input(bmi_output)
        count_vec = bmi.binner.output
        sum_count = count_vec.sum(axis=0)
        if sum_count[neuron_id] >= threshold:
            frFlag = 1
    if mode != "firing rate":
        spike_id = bmi_output.spk_id
        if spike_id == targetID:
            spkFlag = 1
        
    if frFlag == 1 or spkFlag == 1:
        send_signal_to_teensy(bmi)
    else:
        reset_signal_to_teensy(bmi)

In [9]:
# prb file
prb_path = "prb_a2x32.prb"
prb = probe()
prb.load(prb_path)
# prb.show()

ttlport = '/dev/ttyACM0'

In [10]:
# bmi
def bmi_realtime_laser(bmi,bsize,Bbins,neuron_id,threshold,t_smooth
                       ,bmi_update_rule,posterior_threshold,
                      two_steps_decoding,mode,targetID):
   
    pos_buffer_len = int(t_smooth / bsize)
    

    bmi.bmi_update_rule = bmi_update_rule
    bmi.posterior_threshold = posterior_threshold
    bmi.pos_buffer_len = pos_buffer_len # position buffer length for moving average
    bmi.two_steps = two_steps_decoding
    BMI.set_binner(bmi,bin_size = bsize ,B_bins = Bbins)


    if bmi.binner is not None:
        try:
            
# annotated parts are made into functions
#             while True:
#                 bmi_output = bmi.read_bmi()
#                 bmi.binner.input(bmi_output)
#                 count_vec = bmi.binner.output
#                 sum_count = count_vec.sum(axis=0)
#         #         print(sum_count[neuron_id])
#                 if sum_count[neuron_id] >= threshold:
#                     send_signal_to_teensy()
#                 else:
#                     reset_signal_to_teensy()

            bmi_loop(lambda : bmi_func(bmi,mode,targetID,neuron_id,threshold))
        
        except KeyboardInterrupt:
            print("Terminating the loop...")
        finally:
            bmi.close() # close bmi


In [15]:
bmi = BMI(prb=prb, fetfile=fet_path, ttlport=ttlport)
print(np.unique(spk_id))

32 groups on probe
6 groups is configured in the FPGA: [ 4  5  6 11 12 15]
7 neurons are configured in the FPGA
---1. BMI spike-model initiation succeed---

spike-id packet channel is opened

spike-id and feature is saved to fet.bin

[0 1 2 3 4 5 6]


In [73]:
# function call parts
# you can edit the values if you need
# set var

# bsize = 100e-3 # 0.1 sec
bsize = 0.1
Bbins = 10
neuron_id = 4
threshold = 125
t_smooth = 3

bmi_update_rule = 'moving_average'
posterior_threshold = 0.01
two_steps_decoding = False

#================#
#      mode      #
# 1. firing rate #
#   2. spike ID  #
#     3. both    #
#================#

mode = 'both'
targetID = 4

bmi_realtime_laser(bmi,bsize,Bbins,neuron_id,threshold,t_smooth
                       ,bmi_update_rule,posterior_threshold,
                      two_steps_decoding,mode,targetID)

32 groups on probe
6 groups is configured in the FPGA: [ 4  5  6 11 12 15]
7 neurons are configured in the FPGA
---1. BMI spike-model initiation succeed---

spike-id packet channel is opened

spike-id and feature is saved to fet.bin

BMI binner: 10 bins 7 units, each bin is 0.1 seconds
---2. BMI binner initiation succeed---

Terminating the loop...


In [74]:
# count_vec
bmi.binner.output

array([[945.,  13.,   0.,  12.,   9.,   5.,   0.],
       [944.,  14.,   1.,  12.,   9.,   6.,   1.],
       [954.,  12.,   0.,  14.,  12.,   6.,   0.],
       [961.,  12.,   0.,  12.,  11.,   7.,   0.],
       [946.,  12.,   0.,  14.,  11.,   7.,   1.],
       [962.,  14.,   0.,   9.,   5.,   6.,   0.],
       [955.,  13.,   1.,  12.,   8.,   4.,   1.],
       [958.,  15.,   0.,  13.,   5.,   7.,   1.],
       [951.,  14.,   1.,  11.,   7.,   5.,   3.],
       [532.,   6.,   0.,   4.,   4.,   3.,   1.]])

In [75]:
bmi.fpga.configured_groups

array([ 4,  5,  6, 11, 12, 15])

In [12]:
print(bmi.TTLserial)

Serial<id=0x7f922132bf70, open=False>(port='/dev/ttyACM1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0, xonxoff=False, rtscts=False, dsrdtr=False)


# DataFrame

In [ ]:
# fet to dataframe
fet_df = pd.DataFrame(fet,columns = ['t','grp_id','fets_0','fets_1','fets_2','fets_3','spk_id','energy'])
fet_df.head(10)

In [ ]:
np.unique(spk_id)

In [ ]:
# visualization time(spk_id)
plt.vlines(t[spk_id==4], 0, 1)
plt.xlim([0, 100000])

In [ ]:
# counts at each time
unique_times, counts = np.unique(t[spk_id==1], return_counts=True)
counts